# Amazon Bedrock Guardrails
Explore content filtering, PII redaction, topic blocking, and word filters

In [6]:
import boto3
import json

REGION = 'us-east-1'
bedrock = boto3.client('bedrock', region_name=REGION)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=REGION)

print(f"Region: {REGION}")

Region: us-east-1


## 1. Create Basic Guardrail with Content Filters

In [7]:
response = bedrock.create_guardrail(
    name='content-filter-guardrail',
    description='Filter harmful content',
    contentPolicyConfig={
        'filtersConfig': [
            {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'MISCONDUCT', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'}
        ]
    },
    blockedInputMessaging='Your input was blocked due to content policy.',
    blockedOutputsMessaging='The response was blocked due to content policy.'
)

GUARDRAIL_ID = response['guardrailId']
GUARDRAIL_VERSION = response['version']

print(f"✓ Created guardrail: {GUARDRAIL_ID}")
print(f"  Version: {GUARDRAIL_VERSION}")

ConflictException: An error occurred (ConflictException) when calling the CreateGuardrail operation: Another guardrail in your account already has this name. Retry your request with a different name.

## 2. Test Content Filtering

In [8]:
def test_with_guardrail(prompt, guardrail_id, guardrail_version):
    try:
        response = bedrock_runtime.converse(
            modelId='us.amazon.nova-lite-v1:0',
            messages=[{'role': 'user', 'content': [{'text': prompt}]}],
            guardrailConfig={
                'guardrailIdentifier': guardrail_id,
                'guardrailVersion': guardrail_version
            }
        )
        
        if response['stopReason'] == 'guardrail_intervened':
            return {'blocked': True, 'reason': 'Guardrail intervention'}
        
        return {'blocked': False, 'response': response['output']['message']['content'][0]['text']}
    except Exception as e:
        return {'blocked': True, 'error': str(e)}

# Test safe prompt
result = test_with_guardrail("What is machine learning?", GUARDRAIL_ID, GUARDRAIL_VERSION)
print(f"Safe prompt: {result}\n")

# Test potentially harmful prompt
result = test_with_guardrail("How to hack a computer?", GUARDRAIL_ID, GUARDRAIL_VERSION)
print(f"Harmful prompt: {result}")

Safe prompt: {'blocked': False, 'response': 'Machine learning is a subset of artificial intelligence (AI) that focuses on developing algorithms and statistical models that enable computers to perform tasks without explicit instructions. Instead, these systems learn from data, identify patterns, and make decisions based on the information they have processed.\n\nHere are some key aspects of machine learning:\n\n1. **Types of Machine Learning**:\n   - **Supervised Learning**: The model is trained on a labeled dataset, which means that each training example is paired with an output label. The goal is to learn a mapping from inputs to outputs.\n   - **Unsupervised Learning**: The model is trained on a dataset without labeled responses. The system tries to learn the underlying structure or distribution in the data.\n   - **Semi-Supervised Learning**: A combination of both supervised and unsupervised learning, using a small amount of labeled data and a large amount of unlabeled data.\n   - *

## 3. Create Guardrail with PII Redaction

In [9]:
response = bedrock.create_guardrail(
    name='pii-redaction-guardrail',
    description='Redact sensitive PII',
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'BLOCK'},
            {'type': 'PHONE', 'action': 'BLOCK'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'ADDRESS', 'action': 'ANONYMIZE'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'}
        ]
    },
    blockedInputMessaging='Input contains sensitive information.',
    blockedOutputsMessaging='Output contains sensitive information.'
)

PII_GUARDRAIL_ID = response['guardrailId']
PII_GUARDRAIL_VERSION = response['version']

print(f"✓ Created PII guardrail: {PII_GUARDRAIL_ID}")

ConflictException: An error occurred (ConflictException) when calling the CreateGuardrail operation: Another guardrail in your account already has this name. Retry your request with a different name.

## 4. Test PII Redaction

In [10]:
# Test with PII
prompts_with_pii = [
    "My email is john.doe@example.com and phone is 555-1234",
    "John Smith lives at 123 Main Street",
    "My credit card is 4532-1234-5678-9010"
]

for prompt in prompts_with_pii:
    result = test_with_guardrail(prompt, PII_GUARDRAIL_ID, PII_GUARDRAIL_VERSION)
    print(f"Input: {prompt}")
    print(f"Result: {result}\n")

Input: My email is john.doe@example.com and phone is 555-1234
Result: {'blocked': True, 'reason': 'Guardrail intervention'}

Input: John Smith lives at 123 Main Street
Result: {'blocked': False, 'response': ' - The generated text has been blocked by our content filters.'}

Input: My credit card is 4532-1234-5678-9010
Result: {'blocked': True, 'reason': 'Guardrail intervention'}



## 5. Create Guardrail with Topic Blocking

In [11]:
response = bedrock.create_guardrail(
    name='topic-blocking-guardrail',
    description='Block specific topics',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Financial Advice',
                'definition': 'Investment advice, stock tips, financial planning',
                'examples': [
                    'Should I invest in stocks?',
                    'What stocks should I buy?',
                    'Give me investment advice'
                ],
                'type': 'DENY'
            },
            {
                'name': 'Medical Advice',
                'definition': 'Medical diagnosis, treatment recommendations, health advice',
                'examples': [
                    'What medication should I take?',
                    'How do I treat this condition?',
                    'Diagnose my symptoms'
                ],
                'type': 'DENY'
            }
        ]
    },
    blockedInputMessaging='This topic is not allowed.',
    blockedOutputsMessaging='Cannot provide information on this topic.'
)

TOPIC_GUARDRAIL_ID = response['guardrailId']
TOPIC_GUARDRAIL_VERSION = response['version']

print(f"✓ Created topic guardrail: {TOPIC_GUARDRAIL_ID}")

✓ Created topic guardrail: uppmittcs5wj


## 6. Test Topic Blocking

In [12]:
test_prompts = [
    "What is machine learning?",  # Allowed
    "Should I invest in Bitcoin?",  # Blocked - financial
    "What medication should I take for headache?",  # Blocked - medical
    "Explain cloud computing"  # Allowed
]

for prompt in test_prompts:
    result = test_with_guardrail(prompt, TOPIC_GUARDRAIL_ID, TOPIC_GUARDRAIL_VERSION)
    print(f"Prompt: {prompt}")
    print(f"Result: {result}\n")

Prompt: What is machine learning?
Result: {'blocked': False, 'response': "Machine learning is a subset of artificial intelligence (AI) that focuses on developing algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data. Here are some key points about machine learning:\n\n1. **Definition**: Machine learning is the process of training a computer to learn from data, identify patterns, and make decisions with minimal human intervention.\n\n2. **Types of Machine Learning**:\n   - **Supervised Learning**: The model is trained on a labeled dataset, which means that each training example is paired with an output label. Common tasks include classification and regression.\n   - **Unsupervised Learning**: The model is trained on a dataset without labeled responses. The system tries to learn the patterns and structure from the input data on its own. Common tasks include clustering and association.\n   - **Semi-Supervised Learning**: A co

## 7. Create Guardrail with Word Filters

In [13]:
response = bedrock.create_guardrail(
    name='word-filter-guardrail',
    description='Block specific words and phrases',
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'competitor'},
            {'text': 'confidential'},
            {'text': 'internal only'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    blockedInputMessaging='Input contains blocked words.',
    blockedOutputsMessaging='Output contains blocked words.'
)

WORD_GUARDRAIL_ID = response['guardrailId']
WORD_GUARDRAIL_VERSION = response['version']

print(f"✓ Created word filter guardrail: {WORD_GUARDRAIL_ID}")

✓ Created word filter guardrail: c2wrrpe5iypx


## 8. Test Word Filters

In [14]:
test_prompts = [
    "Tell me about AWS services",  # Allowed
    "What about our competitor's product?",  # Blocked
    "This is confidential information",  # Blocked
    "Explain cloud architecture"  # Allowed
]

for prompt in test_prompts:
    result = test_with_guardrail(prompt, WORD_GUARDRAIL_ID, WORD_GUARDRAIL_VERSION)
    print(f"Prompt: {prompt}")
    print(f"Result: {result}\n")

Prompt: Tell me about AWS services
Result: {'blocked': False, 'response': 'Amazon Web Services (AWS) offers a wide range of cloud computing services that can be used to build and operate applications and websites. Here are some of the key services:\n\n### Compute Services\n1. **Amazon EC2 (Elastic Compute Cloud)**\n   - Scalable virtual servers to run applications.\n   \n2. **AWS Lambda**\n   - Serverless compute service that runs code in response to events.\n\n3. **AWS Fargate**\n   - Serverless compute engine for containers.\n\n4. **AWS Batch**\n   - Batch computing service for running large-scale, parallel workloads.\n\n### Storage Services\n1. **Amazon S3 (Simple Storage Service)**\n   - Object storage service for storing and retrieving any amount of data.\n\n2. **Amazon EBS (Elastic Block Store)**\n   - Block storage for use with EC2 instances.\n\n3. **Amazon EFS (Elastic File System)**\n   - Scalable file storage service for use with EC2.\n\n4. **AWS Glacier**\n   - Low-cost clou

## 9. Create Comprehensive Guardrail

In [15]:
response = bedrock.create_guardrail(
    name='comprehensive-guardrail',
    description='All protections enabled',
    contentPolicyConfig={
        'filtersConfig': [
            {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'MISCONDUCT', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'BLOCK'},
            {'type': 'PHONE', 'action': 'BLOCK'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ]
    },
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Financial Advice',
                'definition': 'Investment and financial planning advice',
                'examples': ['Should I invest?', 'Stock recommendations'],
                'type': 'DENY'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [{'text': 'confidential'}],
        'managedWordListsConfig': [{'type': 'PROFANITY'}]
    },
    blockedInputMessaging='Your input was blocked by our safety policies.',
    blockedOutputsMessaging='The response was blocked by our safety policies.'
)

COMP_GUARDRAIL_ID = response['guardrailId']
COMP_GUARDRAIL_VERSION = response['version']

print(f"✓ Created comprehensive guardrail: {COMP_GUARDRAIL_ID}")

✓ Created comprehensive guardrail: n1zy4fcq73cn


## 10. Use Guardrail with Knowledge Base

In [16]:
# Load KB config if available
try:
    with open('kb_config.json', 'r') as f:
        kb_config = json.load(f)
    
    KB_ID = kb_config['knowledge_base_id']
    
    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)
    
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={'text': 'What equipment do we have?'},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{REGION}::foundation-model/amazon.nova-pro-v1:0',
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': COMP_GUARDRAIL_ID,
                        'guardrailVersion': COMP_GUARDRAIL_VERSION
                    }
                }
            }
        }
    )
    
    print("✓ KB query with guardrail:")
    print(response['output']['text'])
except FileNotFoundError:
    print("KB config not found - run knowledge base notebook first")

✓ KB query with guardrail:
Based on the retrieved results, the equipment available includes:

- **LE950 Large Excavator**:
  - General Duty Buckets - Heavy Duty Buckets - Severe Duty Buckets - Extreme Duty Buckets - Hydraulic Hammers - Multi-Processors - Rippers - Thumbs - **FL250**:
  - Standard Forks These are the specific attachments and tools mentioned for the LE950 and FL250 equipment.


## 11. List and Manage Guardrails

In [17]:
# List all guardrails
response = bedrock.list_guardrails()

print("Your Guardrails:")
for guardrail in response['guardrails']:
    print(f"  - {guardrail['name']} ({guardrail['id']}) v{guardrail['version']}")

# Get guardrail details
response = bedrock.get_guardrail(
    guardrailIdentifier=COMP_GUARDRAIL_ID,
    guardrailVersion=COMP_GUARDRAIL_VERSION
)

print(f"\nComprehensive Guardrail Details:")
print(json.dumps(response, indent=2, default=str))

Your Guardrails:
  - training-guardrail (55hw5kof04i1) vDRAFT
  - pii-redaction-guardrail (aevrweb1his0) vDRAFT
  - word-filter-guardrail (c2wrrpe5iypx) vDRAFT
  - merxbp-realestate-guardrail (f0sxd7fpx6mw) vDRAFT
  - content-filter-guardrail (jkcm1dbq0zrr) vDRAFT
  - sql-guardrail (jvq909rujqzs) vDRAFT
  - comprehensive-guardrail (n1zy4fcq73cn) vDRAFT
  - rupal-trip-planning (tuqjju9th0ry) vDRAFT
  - topic-blocking-guardrail (uppmittcs5wj) vDRAFT

Comprehensive Guardrail Details:
{
  "ResponseMetadata": {
    "RequestId": "301a5c35-395a-4942-a3fc-de5ef8d53671",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 09 Dec 2025 06:35:34 GMT",
      "content-type": "application/json",
      "content-length": "2528",
      "connection": "keep-alive",
      "x-amzn-requestid": "301a5c35-395a-4942-a3fc-de5ef8d53671"
    },
    "RetryAttempts": 0
  },
  "name": "comprehensive-guardrail",
  "description": "All protections enabled",
  "guardrailId": "n1zy4fcq73cn",
  "guardrailAr

## 12. Save Guardrail Configuration

In [18]:
guardrail_config = {
    'content_filter': {'id': GUARDRAIL_ID, 'version': GUARDRAIL_VERSION},
    'pii_redaction': {'id': PII_GUARDRAIL_ID, 'version': PII_GUARDRAIL_VERSION},
    'topic_blocking': {'id': TOPIC_GUARDRAIL_ID, 'version': TOPIC_GUARDRAIL_VERSION},
    'word_filter': {'id': WORD_GUARDRAIL_ID, 'version': WORD_GUARDRAIL_VERSION},
    'comprehensive': {'id': COMP_GUARDRAIL_ID, 'version': COMP_GUARDRAIL_VERSION}
}

with open('guardrail_config.json', 'w') as f:
    json.dump(guardrail_config, f, indent=2)

print("✓ Guardrail config saved to guardrail_config.json")
print(json.dumps(guardrail_config, indent=2))

✓ Guardrail config saved to guardrail_config.json
{
  "content_filter": {
    "id": "jkcm1dbq0zrr",
    "version": "DRAFT"
  },
  "pii_redaction": {
    "id": "aevrweb1his0",
    "version": "DRAFT"
  },
  "topic_blocking": {
    "id": "uppmittcs5wj",
    "version": "DRAFT"
  },
  "word_filter": {
    "id": "c2wrrpe5iypx",
    "version": "DRAFT"
  },
  "comprehensive": {
    "id": "n1zy4fcq73cn",
    "version": "DRAFT"
  }
}


## 13. Cleanup (Optional)

In [14]:
# Uncomment to delete guardrails
for guardrail_id in [GUARDRAIL_ID, PII_GUARDRAIL_ID, TOPIC_GUARDRAIL_ID, WORD_GUARDRAIL_ID, COMP_GUARDRAIL_ID]:
    try:
        bedrock.delete_guardrail(guardrailIdentifier=guardrail_id)
        print(f"✓ Deleted guardrail: {guardrail_id}")
    except Exception as e:
        print(f"Error deleting {guardrail_id}: {e}")

#print("Cleanup skipped - uncomment to delete guardrails")

✓ Deleted guardrail: aw7e7puxo9gk
✓ Deleted guardrail: s410guj3tagg
✓ Deleted guardrail: 025mmxtcw1ck
✓ Deleted guardrail: wu79j3z5ad69
✓ Deleted guardrail: z04w5tryx7d8


## Summary

### Guardrail Types:

1. **Content Filters** - Block hate, violence, sexual, misconduct
2. **PII Redaction** - Block/anonymize sensitive information
3. **Topic Blocking** - Deny specific topics (financial, medical)
4. **Word Filters** - Block specific words and profanity
5. **Comprehensive** - All protections combined

### Filter Strengths:

- **NONE** - No filtering
- **LOW** - Minimal filtering
- **MEDIUM** - Moderate filtering
- **HIGH** - Strict filtering

### PII Actions:

- **BLOCK** - Reject content with PII
- **ANONYMIZE** - Replace PII with placeholders

### Use Cases:

- **Customer Service** - Block harmful content, redact PII
- **Healthcare** - Strict PII protection, medical topic control
- **Financial** - Block financial advice, protect sensitive data
- **Enterprise** - Prevent confidential information leakage

### Integration:

- Direct model invocation (Converse API)
- Knowledge Base queries
- Agent responses
- Streaming responses

### Best Practices:

1. Start with comprehensive guardrail
2. Test with real use cases
3. Adjust filter strengths based on needs
4. Monitor blocked content
5. Update topic definitions regularly
6. Use appropriate PII actions for your use case